## Setting(오류 방지를 위해 실행 권장)

In [ ]:
# # === Run this ONCE at the very top of your notebook ===
# import os, warnings

# # transformers/torch의 각종 경고 잠재우기
# os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"   # HF 권고/Deprecation 메시지 억제
# os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"        # 진행바 억제(선택)
# os.environ["PYTHONWARNINGS"] = "ignore::FutureWarning"   # 파이썬 FutureWarning 전역 무시
# os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# # 특정 메시지만 골라 무시하고 싶다면(더 보수적)
# warnings.filterwarnings
#     "ignore",
#     category=FutureWarning,
#     message=r"`encoder_attention_mask` is deprecated and will be removed in version 4\.55\.0",
# )

# # HF 로깅 레벨 낮추기(선택)
# try:
#     from transformers.utils import logging as hf_logging
#     hf_logging.set_verbosity_error()  # or set_verbosity_warning()
# except Exception:
#     pass

In [24]:
# from sentence_transformers import CrossEncoder
# import torch

# _ce_model_cache = None
# def ce_predict_pairs(pairs):
#     global _ce_model_cache
#     if not USE_CE:
#         return np.zeros(len(pairs), dtype=float)

#     dev = "cuda" if torch.cuda.is_available() else "cpu"
#     if _ce_model_cache is None:
#         _ce_model_cache = CrossEncoder(
#             CE_MODEL,
#             device=dev,
#             max_length=512,
#             trust_remote_code=True,                 # Jina CE는 custom code 필요
#             model_kwargs={"attn_implementation":"sdpa"}  # ★ SDPA 사용
#         )
#     return _ce_model_cache.predict(pairs)


In [10]:
# pip install einops

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\ogod3\miniconda3\envs\[gpu]\python.exe -m pip install --upgrade pip


In [12]:
# pip install hf_xet

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------------------ --------------------- 1.3/2.8 MB 6.7 MB/s eta 0:00:01
   ------------------------------------- -- 2.6/2.8 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 6.0 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\ogod3\miniconda3\envs\[gpu]\python.exe -m pip install --upgrade pip


In [32]:
# pip install -U accelerate

Note: you may need to restart the kernel to use updated packages.Collecting accelerate



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\ogod3\miniconda3\envs\[gpu]\python.exe -m pip install --upgrade pip


## Model load

In [2]:
from tqdm import tqdm
import json
import pandas as pd

data_dir = os.path.join("..", "..", "data", "cleaned")
file_list = sorted(glob.glob(os.path.join(data_dir, "dataon_clean_part*.jsonl")))

df_list = []
for f in file_list:
    print(f"📂 Loading {f}")
    rows = []
    with open(f, "r", encoding="utf-8") as fp:
        for line in tqdm(fp, desc=f"Reading {os.path.basename(f)}"):
            rows.append(json.loads(line))
    df_list.append(pd.DataFrame(rows))

clean = pd.concat(df_list, ignore_index=True)
print(f"✅ Loaded total {len(clean)} records from {len(file_list)} files.")

📂 Loading ..\..\data\cleaned\dataon_clean_part1.jsonl


Reading dataon_clean_part1.jsonl: 309086it [00:19, 15977.82it/s]


📂 Loading ..\..\data\cleaned\dataon_clean_part2.jsonl


Reading dataon_clean_part2.jsonl: 287104it [00:19, 14512.63it/s]


📂 Loading ..\..\data\cleaned\dataon_clean_part3.jsonl


Reading dataon_clean_part3.jsonl: 194834it [00:27, 7109.66it/s] 


📂 Loading ..\..\data\cleaned\dataon_clean_part4.jsonl


Reading dataon_clean_part4.jsonl: 155819it [00:19, 7924.52it/s] 


✅ Loaded total 946843 records from 4 files.


In [3]:
clean.value_counts(['lang'])

lang 
en       499714
de        81542
it        72455
ca        68521
nl        41296
es        33918
ro        24058
fr        23166
cs        18027
sk        15872
et        15742
da         9882
pt         7874
sl         7720
sv         7145
ko         3980
pl         3298
lt         2490
el         1801
hu         1600
lv         1149
no         1038
hr         1035
id          843
fi          597
mk          443
bg          403
ja          278
af          203
tl          172
uk          117
zh-cn       106
vi           83
so           75
cy           45
bn           45
tr           42
ru           29
sw           22
sq            9
he            7
th            1
dtype: int64

In [4]:
# df: pandas DataFrame
mask = clean['lang'].astype(str).str.strip().str.lower().eq('en')
clean = clean.loc[mask].reset_index(drop=True)

In [5]:
clean.value_counts(['lang'])

lang
en      499714
dtype: int64

In [7]:
paper_path = os.path.join("..", "..", "data", "cleaned", "papers_clean.jsonl")
paper = pd.read_json(paper_path, lines=True)
paper.head()

,id,title,description,url,year,keyword,score,reason,level
0,DIKO0015020310,Schizosaccharomyces pombe의 chronological life ...,Chronological life span을 측정하는 전통적인 방법은 station...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2019,span,NaN,NaN,NaN
1,NART10668840,A space with infinite span and zero surjective...,Abstract Topics about continua (compact connec...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2001,span,NaN,NaN,NaN
2,NART110393231,"T-Span, T-Edge Span Critical Graphs",Given a graph and a finite set of positive int...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2019,span,NaN,NaN,NaN
3,NART111038554,Nutrients and Pathways that Regulate Health Sp...,Both life span and health span are influenced ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2020,span,NaN,NaN,NaN
4,DIKO0008421821,랭거 아치교의 Rise-Span 비의 최적화,If horizontal reaction generated by vertical l...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2000,span,NaN,NaN,NaN


In [8]:
paper.isnull().sum()

id                 0
title              0
description        0
url                0
year               0
keyword            0
score          24073
reason         24073
level          24073
dtype: int64

## 전처리(결측치 삭제)

In [10]:
# %% [markdown]
# 전처리 1단계: 결측치 제거 + 중복 제거 (JSONL 버전)

# %%
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

# 입력/출력 경로 (Modeling.ipynb 위치 기준)
SRC = os.path.join("..", "..", "data", "cleaned", "papers_clean.jsonl")
DST = os.path.join("..", "..", "data", "cleaned", "papers_clean.prep.csv")

# 1) 로드 (진행률 표시)
rows = []
with open(SRC, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="📄 Loading papers_clean.jsonl"):
        rows.append(json.loads(line))
df = pd.DataFrame(rows)
print(f"✅ Loaded {len(df)} rows")

# 2) 컬럼명 소문자로 통일(혼용 대비)
df = df.rename(columns={c: c.lower() for c in df.columns})

# 3) 결측치로 간주할 문자열 패턴들 정의
NULL_MARKERS = {"", " ", "nan", "none", "null", "-", "--"}
def normalize_nulls(s):
    if pd.isna(s):
        return np.nan
    s2 = str(s).strip()
    return np.nan if s2.lower() in NULL_MARKERS or len(s2) == 0 else s2

# 4) title/description/url 기본 정리
for col in ["title", "description", "url"]:
    if col in df.columns:
        df[col] = df[col].map(normalize_nulls)
    else:
        df[col] = np.nan

before = len(df)

# 5) description 결측 행 제거
df = df.dropna(subset=["description"])

# 6) 중복 행 제거 (title 기준)
df = df.drop_duplicates(subset=["title"])

# (선택) title이 비어있으면 제거
# df = df.dropna(subset=["title"])

after = len(df)
removed = before - after

# 7) 저장
df.to_csv(DST, index=False, encoding="utf-8-sig")

print(f"전처리 완료: {removed}행 제거 → {after}행 남김")
print(f"저장 위치: {os.path.abspath(DST)}")

# 필요시 미리보기
# display(df.head())

📄 Loading papers_clean.jsonl: 24073it [00:00, 74568.39it/s]


✅ Loaded 24073 rows
전처리 완료: 783행 제거 → 23290행 남김
저장 위치: c:\Users\juni4\OneDrive\바탕 화면\data\researchguide-ai\data\cleaned\papers_clean.prep.csv


In [11]:
# %% [markdown]
# 전처리 1단계: 결측치 제거 + 중복 제거 (DataON JSONL 통합 버전)

# %%
import pandas as pd
import numpy as np
import os
import glob
import json
from tqdm import tqdm

# 입력/출력 경로
DATA_DIR = os.path.join("..", "..", "data", "cleaned")  # src/Modeling → data/cleaned
SRC_PATTERN = os.path.join(DATA_DIR, "dataon_clean_part*.jsonl")
DST1 = os.path.join(DATA_DIR, "datasets_clean_prep.csv")

# 1) 여러 JSONL 파일 로드
file_list = sorted(glob.glob(SRC_PATTERN))
df_list = []

for f in file_list:
    print(f"📂 Loading {os.path.basename(f)}")
    rows = []
    with open(f, "r", encoding="utf-8") as fp:
        for line in tqdm(fp, desc=f"Reading {os.path.basename(f)}"):
            rows.append(json.loads(line))
    df_list.append(pd.DataFrame(rows))

df1 = pd.concat(df_list, ignore_index=True)
print(f"✅ Loaded total {len(df1)} records from {len(file_list)} files")

# 2) 컬럼명 소문자로 통일(혼용 대비)
df1 = df1.rename(columns={c: c.lower() for c in df1.columns})

# 3) 결측치로 간주할 문자열 패턴들 정의
NULL_MARKERS = {"", " ", "nan", "none", "null", "-", "--"}
def normalize_nulls(s):
    if pd.isna(s):
        return np.nan
    s2 = str(s).strip()
    return np.nan if s2.lower() in NULL_MARKERS or len(s2) == 0 else s2

# 4) title/description/url 기본 정리(공백 정리 + NULL_MARKERS → NaN)
for col in ["title", "description", "url"]:
    if col in df1.columns:
        df1[col] = df1[col].map(normalize_nulls)
    else:
        df1[col] = np.nan

before1 = len(df1)

# 5) description 결측 행 제거
df1 = df1.dropna(subset=["description"])

# 6) 중복 행 제거 (title 기준)
df1 = df1.drop_duplicates(subset=["title"])

after1 = len(df1)
removed1 = before1 - after1

# 7) 저장
df1.to_csv(DST1, index=False, encoding="utf-8-sig")

print(f"전처리 완료: {removed1}행 제거 → {after1}행 남김")
print(f"저장 위치: {os.path.abspath(DST1)}")

📂 Loading dataon_clean_part1.jsonl


Reading dataon_clean_part1.jsonl: 309086it [00:36, 8462.80it/s] 


📂 Loading dataon_clean_part2.jsonl


Reading dataon_clean_part2.jsonl: 287104it [00:32, 8832.67it/s] 


📂 Loading dataon_clean_part3.jsonl


Reading dataon_clean_part3.jsonl: 194834it [00:30, 6418.48it/s] 


📂 Loading dataon_clean_part4.jsonl


Reading dataon_clean_part4.jsonl: 155819it [00:18, 8627.13it/s] 


✅ Loaded total 946843 records from 4 files
전처리 완료: 604547행 제거 → 342296행 남김
저장 위치: c:\Users\juni4\OneDrive\바탕 화면\data\researchguide-ai\data\cleaned\datasets_clean_prep.csv


In [12]:
# 파일 경로 수정: src/Modeling → data/cleaned
prep_path = os.path.join("..", "..", "data", "cleaned", "papers_clean.prep.csv")

# CSV 읽기
prep = pd.read_csv(prep_path, encoding="utf-8-sig")

prep.head()

,id,title,description,url,year,keyword,score,reason,level
0,DIKO0015020310,Schizosaccharomyces pombe의 chronological life ...,Chronological life span을 측정하는 전통적인 방법은 station...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2019,span,NaN,NaN,NaN
1,NART10668840,A space with infinite span and zero surjective...,Abstract Topics about continua (compact connec...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2001,span,NaN,NaN,NaN
2,NART110393231,"T-Span, T-Edge Span Critical Graphs",Given a graph and a finite set of positive int...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2019,span,NaN,NaN,NaN
3,NART111038554,Nutrients and Pathways that Regulate Health Sp...,Both life span and health span are influenced ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2020,span,NaN,NaN,NaN
4,DIKO0008421821,랭거 아치교의 Rise-Span 비의 최적화,If horizontal reaction generated by vertical l...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,2000,span,NaN,NaN,NaN


In [13]:
# 파일 경로 지정 (src/Modeling → data/cleaned)
data_path = os.path.join("..", "..", "data", "cleaned", "datasets_clean_prep.csv")

# CSV 파일 읽기
data = pd.read_csv(data_path, encoding="utf-8-sig")

data.head()

C:\Users\juni4\AppData\Local\Temp\ipykernel_1984\489409384.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path, encoding="utf-8-sig")


,id,title,description,keywords,org,year,url,doi,lang
0,de776da1a1d0f302eeb7608c32c810d3,This item has been removed from publication,En este archivo se encuentran los resultados d...,"['', '']",OpenAIRE,9999,https://dx.doi.org/10.71590/rsfwiq/0yfh7w,NaN,es
1,18fce9b918b6abc277517b4a065df9e5,Homemade Toys,Supported by funding from the Department of Ar...,"['', 'Toys;Bréagáin;An Baile Dubh;Ballyduff']",OpenAIRE,7938,https://dx.doi.org/10.7925/drs1.duchas_4504414,NaN,en
2,a8331499d9a5e7796fdfa528657b7143,Local Forge,Supported by funding from the Department of Ar...,"['', 'Smithing;Gaibhneacht']",OpenAIRE,4938,https://dx.doi.org/10.7925/drs1.duchas_4927495,NaN,en
3,e81bb07a6b450ac61e926e8b27d0958e,Agta hunter-gatherer oral microbiomes are shap...,Here we investigate the effects of extensive s...,"['', 'Hunter-gatherers;Oral microbiome;Disease...",OpenAIRE,2030,https://dx.doi.org/10.5281/zenodo.6338839;http...,NaN,en
4,03a0e4393dd46936b3b5d62d3a902860,Local Fairs,Supported by funding from the Department of Ar...,"['', 'Commerce;Buying and selling;Díol agus ce...",OpenAIRE,2029,https://dx.doi.org/10.7925/drs1.duchas_5191737,NaN,en


## SBERT 사용을 위한 numpy 다운그레이드, tensorflow 삭제 및 pytorch 설치

In [ ]:
import os, sys

# TF 강제 로딩 방지 (sentence-transformers는 PyTorch만 필요)
os.environ["TRANSFORMERS_NO_TF"] = "1"

# (권장) pip 업데이트
!{sys.executable} -m pip install -U pip

# (권장) NumPy 호환 버전으로 고정
!{sys.executable} -m pip install --upgrade --force-reinstall "numpy==1.26.4"

# (권장) TF 계열 제거 — SBERT에는 불필요하고 종종 충돌 유발
!{sys.executable} -m pip uninstall -y tensorflow tensorflow-gpu tensorboard keras || echo "no tf to remove"

# PyTorch (CUDA 12.9 전용)
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129

# Sentence-Transformers
!{sys.executable} -m pip install -U sentence-transformers

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/b7/3f/945ef7ab14dc4f9d7f40288d2df998d1837ee0888ec3659c813487572faa/pip-25.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.8 MB 5.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.8 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ------------------- -------------------- 7.9/15.8 MB 40.4 MB/s eta 0:00:01
   ---------------------------------------  15.7/15.8 MB 41.2 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 30.1 MB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.


Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Successfully uninstalled tensorflow-2.16.1
Found existing installation: tensorboard 2.16.2
Uninstalling tensorboard-2.16.2:
  Successfully uninstalled tensorboard-2.16.2
Found existing installation: keras 3.1.1
Uninstalling keras-3.1.1:
  Successfully uninstalled keras-3.1.1


^C
^C


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires keras>=3.0.0, which is not installed.
tensorflow-intel 2.16.1 requires tensorboard<2.17,>=2.16, which is not installed.


Looking in indexes: https://download.pytorch.org/whl/cu129
   ---------------------------------------- 0.0/3.6 GB ? eta -:--:--
   ---------------------------------------- 0.0/3.6 GB 57.8 MB/s eta 0:01:02
   ---------------------------------------- 0.0/3.6 GB 59.3 MB/s eta 0:01:00
   ---------------------------------------- 0.0/3.6 GB 59.1 MB/s eta 0:01:00
    --------------------------------------- 0.0/3.6 GB 58.1 MB/s eta 0:01:01
    --------------------------------------- 0.1/3.6 GB 58.8 MB/s eta 0:01:00
    --------------------------------------- 0.1/3.6 GB 58.6 MB/s eta 0:01:00
    --------------------------------------- 0.1/3.6 GB 57.5 MB/s eta 0:01:01
   - -------------------------------------- 0.1/3.6 GB 53.8 MB/s eta 0:01:05
   - -------------------------------------- 0.1/3.6 GB 51.4 MB/s eta 0:01:08
   - -------------------------------------- 0.1/3.6 GB 48.2 MB/s eta 0:01:12
   - -------------------------------------- 0.1/3.6 GB 45.9 MB/s eta 0:01:16
   - --------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2025.9.0 which is incompatible.


  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------------------------- -------------- 7.6/12.0 MB 42.7 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 36.9 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 25.0 MB/s  0:00:00
Using cached huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl (320 kB)
   ------------------

In [30]:
# pytorch 기반 cuda 버전 확인

import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version:", torch.version.cuda)
print("gpu:", torch.cuda.get_device_name(0))

torch: 2.8.0+cu129
cuda available: True
cuda version: 12.9
gpu: NVIDIA GeForce RTX 4070 Laptop GPU


## Sentence Transformer & Cross Encoder 모델 설치(폴더 사용 안할거면 hub에서 설치)

In [1]:
from sentence_transformers import SentenceTransformer

model_name = "paraphrase-multilingual-MiniLM-L12-v2"
save_dir = "local_sbert_models/paraphrase-multilingual-MiniLM-L12-v2"

# 모델 다운로드 + 저장
model = SentenceTransformer(model_name)
model.save(save_dir)
print("Saved SBERT model to:", save_dir)

Saved SBERT model to: local_sbert_models/paraphrase-multilingual-MiniLM-L12-v2


In [ ]:
USE_SBERT = True
SBERT_MODEL_NAME_OR_PATH = "../../public/model/paraphrase-multilingual-MiniLM-L12-v2"

# SentenceTransformer(SBERT_MODEL_NAME_OR_PATH)를 사용하면 Hugging Face 허브가 아닌, 로컬 폴더에서 SBERT 모델을 불러옴


In [2]:
from sentence_transformers import SentenceTransformer
import os

save_dir = os.path.abspath("models/paraphrase-multilingual-MiniLM-L12-v2")
os.makedirs(save_dir, exist_ok=True)

m = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
m.save(save_dir)
print("saved to:", save_dir)

saved to: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\models\paraphrase-multilingual-MiniLM-L12-v2


In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(save_dir, local_files_only=True)  # 오프라인 로드 강제

In [3]:
# ↓ 1회만 실행
import os
from huggingface_hub import snapshot_download

MODEL_ID  = "BAAI/bge-reranker-v2-m3"
LOCAL_DIR = os.path.join("models", "bge-reranker-v2-m3")  # 원하는 경로

os.makedirs(LOCAL_DIR, exist_ok=True)

# Windows에서 symlink 경고/문제 피하기: local_dir_use_symlinks=False
snapshot_download(
    repo_id=MODEL_ID,
    local_dir=LOCAL_DIR,
    local_dir_use_symlinks=False,   # ← 복사본 저장(심볼릭링크 X)
    resume_download=True
)

print("Saved to:", os.path.abspath(LOCAL_DIR))

C:\Users\ogod3\miniconda3\envs\[gpu]\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\ogod3\miniconda3\envs\[gpu]\lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

CMTEB-retrieval-bge-zh-v1.5.png:   0%|          | 0.00/51.5k [00:00<?, ?B/s]

BEIR-e5-mistral.png:   0%|          | 0.00/40.2k [00:00<?, ?B/s]

miracl-bge-m3.png:   0%|          | 0.00/52.0k [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

llama-index.png:   0%|          | 0.00/106k [00:00<?, ?B/s]

BEIR-bge-en-v1.5.png:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

Saved to: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\models\bge-reranker-v2-m3


# 모델

## 캐시 초기화

In [25]:
# ==== Global caches ====
_BM25_P = None
_BM25_D = None
_P_DENSE_TEXTS = None
_D_DENSE_TEXTS = None
_P_DENSE_VECS  = None
_D_DENSE_VECS  = None

def _ensure_indexes_and_dense(papers_df, datasets_df, backend):
    """세션 동안 1회만 구축해서 재사용"""
    global _BM25_P, _BM25_D, _P_DENSE_TEXTS, _D_DENSE_TEXTS, _P_DENSE_VECS, _D_DENSE_VECS

    if _BM25_P is None:
        _BM25_P = WeightedBM25(papers_df, FIELD_WEIGHTS)
    if _BM25_D is None:
        _BM25_D = WeightedBM25(datasets_df, FIELD_WEIGHTS)

    if _P_DENSE_VECS is None:
        if "__dense_text__" not in papers_df.columns:
            papers_df["__dense_text__"] = [compose_dense_text(r) for _, r in papers_df.iterrows()]
        _P_DENSE_TEXTS = papers_df["__dense_text__"].tolist()
        _P_DENSE_VECS  = backend.encode(_P_DENSE_TEXTS)

    if _D_DENSE_VECS is None:
        if "__dense_text__" not in datasets_df.columns:
            datasets_df["__dense_text__"] = [compose_dense_text(r) for _, r in datasets_df.iterrows()]
        _D_DENSE_TEXTS = datasets_df["__dense_text__"].tolist()
        _D_DENSE_VECS  = backend.encode(_D_DENSE_TEXTS)

def reset_retrieval_cache():
    """코퍼스를 바꾸면 호출해서 캐시 초기화"""
    global _BM25_P, _BM25_D, _P_DENSE_TEXTS, _D_DENSE_TEXTS, _P_DENSE_VECS, _D_DENSE_VECS
    _BM25_P = _BM25_D = _P_DENSE_TEXTS = _D_DENSE_TEXTS = _P_DENSE_VECS = _D_DENSE_VECS = None

## 데이터 준비

In [ ]:
# %% [markdown]
# # 논문/데이터셋 추천 (입력: 제목+설명 직접 타이핑)
# - 입력: papers_clean.csv (columns: title, description, url)
# - 질의 입력: input()으로 제목/설명 별도 입력
# - 임베딩: 기본 TF-IDF(char 2~4그램) / (옵션) 로컬 SBERT
# - 질의 벡터: 제목/설명 임베딩을 가중합(W_TITLE, W_DESC)
# - 출력: 상위 TOPK개 (rank, title, description, url, score, reason, level)
# - 저장: recommendations.csv

# %%
import os
import re
import numpy as np
import pandas as pd
from typing import List, Tuple
from IPython.display import display
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

# -------------------- Config --------------------
PAPERS_CSV    = "../../data/papers_clean.prep.csv"
DATASETS_CSV  = "../../data/datasets_clean_prep.csv"
OUTPUT_PAPERS = "../../outputs/recommendations.papers.csv"
OUTPUT_DATA   = "../../outputs/recommendations.datasets.csv"
TOPK          = 5

# 제목/설명 가중치
W_TITLE = 1.0
W_DESC  = 2.0

USE_SBERT = True
SBERT_MODEL_NAME_OR_PATH = "../../public/model/paraphrase-multilingual-MiniLM-L12-v2"  # 로컬에 받은 경로

## Explanation

In [27]:
# --------------------Explanation----------------------------
def safe_text(x) -> str:
    if pd.isna(x):
        return ""
    return str(x)
def get_tokenizer(USE_SBERT: bool):
    """
    SBERT 모드에서는 토크나이저 skip.
    TF-IDF 모드일 때만 Mecab/Fallback 토크나이저 사용.
    """
    if USE_SBERT:
        # SBERT 내부 토크나이저 사용 → 외부 tokenizer 필요 없음
        def dummy_tok(text: str):
            return []  # build_reason 등에선 공백 리스트 반환
        return dummy_tok, "skip"

# 간단 한국어/영문 공통 토크나이저 + 불용어
_STOP = {"및","과","와","에서","으로","으로의","대한","관련","하는","하기","이다","있는","위한",
         "the","a","an","and","of","to","in","for","on","with","by","about","at","as","is","are"}

_rx = re.compile(r"[가-힣A-Za-z0-9]+")

def _lite_tokens(s: str):
    toks = [t.lower() for t in _rx.findall(s or "")]
    return [t for t in toks if t not in _STOP and len(t) > 1]

def label_by_ranked_score(scores: np.ndarray, i: int) -> str:
    # 상위 5개 안에서 상대 기준 (예: 최상위 0~1 구간 정규화)
    smax, smin = float(scores[0]), float(scores[min(len(scores)-1, 4)])
    denom = max(1e-6, smax - smin)
    rel = (float(scores[i]) - smin) / denom  # 0~1
    if rel >= 0.75: return "강추"
    if rel >= 0.45: return "추천"
    return "참고"

## Embedding backends

In [28]:
# -------------------- Embedding backends --------------------
class EmbeddingBackend:
    def fit(self, texts: List[str]): ...
    def encode(self, texts: List[str]): ...

class SBERTBackend:
    def __init__(self, model_path: str):
        from sentence_transformers import SentenceTransformer
        self.model = SentenceTransformer(model_path)

    def fit(self, texts):  # SBERT는 학습 필요 없음
        pass

    def encode(self, texts):
        vecs = self.model.encode(
            texts,
            batch_size=32,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return vecs

def get_backend():
    if USE_SBERT:
        return SBERTBackend(SBERT_MODEL_NAME_OR_PATH)

## 추출적 근거(Extractive)

In [29]:
# ==== 초경량 추출형 추천사유: 입력과 가장 유사한 '한 문장' ====
import re, numpy as np
from sklearn.metrics.pairwise import cosine_similarity

MAX_REASON_CHARS = 100
_rx_split = re.compile(r"(?<=[.!?。？！])\s+|[\r\n]+|[•\u2022]")

def extractive_reason(q_title: str, q_desc: str,
                      doc_title: str, doc_desc: str,
                      backend, max_chars: int = MAX_REASON_CHARS) -> str:
    """입력(제목+설명)과 후보 설명을 비교해 유사도가 가장 큰 문장 1개를 반환"""
    raw = (doc_desc or "").strip() or (doc_title or "")
    cands = [s.strip() for s in _rx_split.split(raw) if s and len(s.strip()) > 2]
    cands = cands[:5]  # 너무 많은 문장 비교 방지 (속도)
    if not cands:
        s = (doc_title or "").strip()
        return tidy_korean_sentence(s, max_chars)

    # 쿼리/문장 임베딩
    q_text = f"{(q_title or '').strip()} {(q_desc or '').strip()}".strip()
    qv = backend.encode([q_text])
    sv = backend.encode(cands)

    # 코사인 유사도 (정규화 임베딩 가정 → 내적 사용, 실패 시 fallback)
    try:
        from scipy import sparse as _sp
        if hasattr(qv, "shape") and len(getattr(qv, "shape", [])) == 1:
            qv = qv.reshape(1, -1)
        sims = (qv @ (sv.T if not _sp.issparse(sv) else sv.T)).toarray().ravel() \
               if _sp.issparse(sv) else (qv @ sv.T).ravel()
    except Exception:
        sims = cosine_similarity(qv, sv)[0]

    # 최댓값부터 후보 인덱스 정렬
    order = list(np.argsort(-sims))
    best = None
    
    for idx in order[:3]:  # 최상위 3개 중에서 너무 겹치지 않는 문장 선택
        cand = cands[int(idx)]
        para = _light_paraphrase_ko(cand)
        # 원문과 너무 비슷하면(겹침률 0.95↑) 다음 후보 시도
        if _overlap_ratio(para, cand) >= 0.95:
            continue
        best = para
        break
    
    if best is None:
        # 전부 비슷하면 최상위 문장만 가볍게 손질
        best = _light_paraphrase_ko(cands[int(order[0])])
    
    best = re.sub(r"\s+", " ", best).strip()
    return tidy_korean_sentence(best, max_chars)

# ==== 한국어 문장 정리(맞춤법/문장부호 최소 정돈) ====
import re, html, unicodedata

_rx_multi_space = re.compile(r"\s+")
def tidy_korean_sentence(text: str, max_chars: int = 100) -> str:
    t = html.unescape(text or "")                 # &#x00B7; 등 HTML 엔티티 해제
    t = unicodedata.normalize("NFKC", t)          # 전각/호환 문자 정규화

    # 불필요한 제로폭/제어문자 제거
    t = re.sub(r"[\u200B-\u200D\uFEFF]", "", t)

    # 괄호 안/앞뒤 공백 정리
    t = re.sub(r"\(\s+", "(", t)
    t = re.sub(r"\s+\)", ")", t)

    # 구두점 앞 공백 제거, 뒤는 한 칸
    t = re.sub(r"\s+([,\.!?;:)\]])", r"\1", t)
    t = re.sub(r"([,;:])(?=\S)", r"\1 ", t)

    # , . 순서/중복 구두점 정리
    t = re.sub(r",\s*\.", ".", t)
    t = re.sub(r"\.\s*,", ".", t)
    t = re.sub(r"([\.!?,])\1+", r"\1", t)

    # 리스트 점/기호류 가볍게 교정
    t = t.replace("•", "·").replace("・", "·")
    t = re.sub(r"\s*·\s*", "·", t)

    # 다중 공백 정리
    t = _rx_multi_space.sub(" ", t).strip()

    # 길이 제한 및 종결 보정
    t = t[:max_chars].rstrip()
    if not t.endswith(("다", "요", "임", "함", ".", "!", "?")):
        t += "."

    return t

# (), [], {}, <>, 〈〉, 《》, 「」, 『』, 【】, 〔〕 등 1층 괄호 블록 제거
_rx_paren_any = re.compile(r"\s*[\(\[\{<〈《「『【〔]\s*[^)\]\}>〉》」』】〕]{0,200}\s*[\)\]\}>〉》」』】〕]\s*")

def drop_paren_glue(s: str) -> str:
    """괄호 안 문구를 모두 제거하고 공백/구두점 정리"""
    if not s:
        return ""
    t = str(s)

    # 중첩 괄호 대비: 더 이상 치환이 안 될 때까지 반복
    prev = None
    while prev != t:
        prev = t
        t = _rx_paren_any.sub(" ", t)

    # 공백 뭉침/구두점 주변 공백 정리
    t = re.sub(r"\s+", " ", t)
    t = re.sub(r"\s+([,\.!?;:])", r"\1", t)   # 구두점 앞 공백 제거
    t = re.sub(r"([,;:])(?=\S)", r"\1 ", t)   # 구두점 뒤 한 칸
    return t.strip()

# 흔한 서두/표현 치환(아주 보수적)
_REP = [
    (r"^(본\s*연구|이\s*연구|본\s*논문|이\s*논문|본\s*문서|이\s*문서)\s*(는|에서는)\s*", ""),  # 서두 삭제
    (r"다룬다", "분석한다"),
    (r"보여준다", "확인했다"),
    (r"제시한다", "제안한다"),
    (r"탐구한다", "살핀다"),
    (r"효과를\s*보였다", "효과를 확인했다"),
    (r"\s*·\s*", "·"),
]
def _light_paraphrase_ko(s: str) -> str:
    t = drop_paren_glue(s)
    for p, r in _REP:
        t = re.sub(p, r, t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def _overlap_ratio(a: str, b: str) -> float:
    # 토큰 겹침 비율(간단 자카드) – 너무 같으면 다른 문장/치환 시도
    import re
    tok = lambda x: set(w for w in re.findall(r"[가-힣A-Za-z0-9]+", x.lower()) if len(w) > 1)
    ta, tb = tok(a), tok(b)
    return (len(ta & tb) / max(1, len(ta))) if ta else 0.0

## LLM prompting(폐기)

In [15]:
# # ==== LLM 기반 '추천 사유' 생성 셀 (Option A: causal LLM) ====
# import torch, re
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# USE_LLM_REASON = True
# LLM_ID = "models/Qwen2.5-1.5B-Instruct"   # 로컬 폴더 or HF repo id
# MAX_REASON_CHARS = 60                    # 총 글자수 한도(두 문장 이내)

# _llm_pipe = None
# def _load_llm_causal():
#     global _llm_pipe
#     if _llm_pipe is not None:
#         return _llm_pipe

#     tok = AutoTokenizer.from_pretrained(
#         LLM_ID, use_fast=True, trust_remote_code=True
#     )
#     model = AutoModelForCausalLM.from_pretrained(
#         LLM_ID,
#         trust_remote_code=True,
#         torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
#         # device_map="auto"  # ← 가속 패키지 없이도 되도록 제거
#     )
#     if torch.cuda.is_available():
#         model = model.to("cuda")

#     if tok.pad_token_id is None and tok.eos_token_id is not None:
#         tok.pad_token_id = tok.eos_token_id

#     _llm_pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tok,
#         device=(0 if torch.cuda.is_available() else -1),
#     )
#     return _llm_pipe

# # --- few-shot 예시(두 문장·100자 이내로 축약) ---
# FEWSHOTS = [
#     {
#         "query_t": "물 부족 도시를 위한 물 순환경제 : 물 시스템의 물 대사, 에너지, 탄소 분석",
#         "query_d": "파주시 물 대사를 통해 재이용·에너지·탄소 상호작용을 정량 분석하고 전력 탈탄소화 필요를 제시.",
#         "doc_t": "Toward Carbon-Neutral Water Systems: Insights from Global Cities",
#         "doc_sents": [
#             "여러 도시의 상하수도 부문에서 온실가스 감축 사례와 부문 간 협력 전략을 제시한다."
#         ],
#         "why": "도시 물 시스템의 탄소감축 전략을 실증해 재이용·전력 탈탄소화 필요성과 직접 맞닿습니다."
#     },
#     {
#         "query_t": "코로나19 이후 소비트렌드 변화에 따른 지식재산권 관련 이슈 및 시사점",
#         "query_d": "팬데믹으로 온라인 소비가 급증하고 IP 리스크·위조품 유통이 확대된 변화를 다룸.",
#         "doc_t": "Illicit Trade in Fakes under the COVID-19",
#         "doc_sents": [
#             "전자상거래 급증이 위조품 확산과 단속 과제에 미친 영향을 분석한다."
#         ],
#         "why": "온라인 전환에서 IP 침해 확대라는 인과가 동일해 정책·집행 시사점이 직접 연결됩니다."
#     },
# ]

# def _fewshots_block():
#     lines = []
#     for ex in FEWSHOTS:
#         lines += [
#             "[사용자]",
#             f'입력(제목+설명): "{ex["query_t"]} || {ex["query_d"]}"',
#             f'후보 문서: 제목="{ex["doc_t"]}"',
#             "관련성이 높은 문장:",
#             *[f"- {s}" for s in ex["doc_sents"]],
#             "[에이전트]",
#             ex["why"],
#         ]
#     return "\n".join(lines)

# # 금지 기호 제거 + 두 문장 제한 + 길이 제한 + 마침표 보정 + ‘베끼기’ 완화
# _FORBIDDEN = r"[=\-•·\*\_\+\|\[\]\{\}\(\)~^<>/\\:;]+"  # 금지 기호 정규식
# _SENT_SPLIT = r"(?<=[\.!?])\s+"

# def _truncate_kr(s: str, limit=MAX_REASON_CHARS) -> str:
#     s = re.sub(r"\s+", " ", s).strip()
#     return s[:limit]

# # 금지 기호/문장 분리 정규식은 기존 변수(_FORBIDDEN, _SENT_SPLIT) 그대로 사용
# def _sanitize_reason(
#     text: str,
#     limit: int = MAX_REASON_CHARS,
#     doc_raw: str | None = None,
#     ban_phrases: tuple[str, ...] = (),
# ) -> str:
#     import re
#     # 기호 제거 + 공백 정리
#     text = re.sub(_FORBIDDEN, " ", text)
#     text = re.sub(r"\s+", " ", text).strip()

#     # 문장 후보들
#     sents = [s.strip() for s in re.split(_SENT_SPLIT, text) if s.strip()]
#     # 제목 되풀이/상투어(“…글입니다”)가 아닌 첫 문장 선택
#     def ok(s: str) -> bool:
#         low = s.lower()
#         if any(bp and bp.lower() in low for bp in ban_phrases): return False
#         if low.endswith(("글입니다", "논문입니다", "데이터입니다")): return False
#         return True

#     first = next((s for s in sents if ok(s)), (sents[0] if sents else ""))
#     text = (first or "").strip()[:limit]

#     # 문서 설명을 거의 베껴 쓰면 템플릿으로 교체
#     if doc_raw:
#         def toks(t): 
#             return set(re.findall(r"[가-힣A-Za-z0-9]+", (t or "").lower()))
#         qt, dt = toks(text), toks(doc_raw)
#         if qt and len(qt & dt) / max(1, len(qt)) > 0.85:
#             text = "입력의 문제의식과 후보의 분석 초점이 겹쳐 실제 활용 맥락에서 높은 연관성이 있습니다."
#             text = text[:limit]

#     # 종결 보정
#     if text and not text.endswith(("다","요","임","함",".","!","?")):
#         text += "."
#     return text

# def llm_reason(query_title, query_desc, doc_title, doc_desc, picked_sents=None):
#     """
#     picked_sents: 문서에서 골라낸 1문장(없으면 doc_desc로 임시 추출)
#     """
#     if not USE_LLM_REASON:
#         return None

#     pipe = _load_llm_causal()
#     sys = "너는 국내외 연구데이터에 대한 연관 논문,데이터 추천 에이전트다."

#     # 문서에서 근거 문장 1~2개 추출(없으면 설명에서 임시 분할)
#     picked_sents = (picked_sents or [])[:2]
#     if not picked_sents:
#         tmp = re.split(r"[.!?。？！]\s+", (doc_desc or "").strip())
#         picked_sents = [t for t in tmp if t][:2]

#     # 지시사항 업데이트: 기호 금지, 베끼지 말기, 두 문장·100자 이내
#     prompt = (
#         f"[시스템] {sys}\n"
#         + _fewshots_block() + "\n"
#         + "[사용자]\n"
#         + f'입력(제목+설명): "{query_title} || {query_desc}"\n'
#         + f'후보 문서: 제목="{doc_title}"\n'
#         + "관련성이 높은 문장:\n"
#         + "\n".join(f"- {s}" for s in picked_sents)
#         + "\n\n요구사항:\n"
#         + "1) 입력 데이터와 후보 데이터가 어떤 내용이 비슷한지 설명하고 설명 열에 있는 내용을 그대로 옮기지 말 것.\n"
#         + "2) 키워드 나열 금지, 특정 부분에서 의미가 비슷하다는 내용으로 서술.\n"
#         + "3) 기호(=, -, :, •, *, _, / 등) 사용 금지.\n"
#         + "4) 한국어로 한 문장 이내, 총 60자 이내로 간결히 작성. 마지막은 마침표로 끝낼 것.\n"
#         + "5) 입력/후보 ‘제목’을 반복하거나 글 전체 요약을 하지 말고, 입력의 A와 후보의 B가 어떻게 ‘의미적으로 대응’하는지 한 가지 포인트만 말할 것.\n"
#         + "[에이전트]\n"
#     )

#     out = pipe(
#         prompt,
#         max_new_tokens=80,
#         do_sample=False,
#         temperature=0.0,
#         top_p=1.0,
#         repetition_penalty=1.05,
#         eos_token_id=pipe.tokenizer.eos_token_id,
#     )[0]["generated_text"]

#     ans_raw = out.split("[에이전트]")[-1].strip()
#     doc_raw  = f"{doc_title} {doc_desc}"
#     ans = _sanitize_reason(
#         ans_raw,
#         MAX_REASON_CHARS,
#         doc_raw=doc_raw,
#         ban_phrases=(query_title, doc_title, "입니다", "글입니다"),
#     )

#     # 너무 짧게 남으면 안전 템플릿
#     if len(ans) < 8:
#         qt = (_key_terms(f"{query_title} {query_desc}", topn=1) or ["핵심 주제"])[0]
#         dt = (_key_terms(doc_desc, topn=1) or ["동일 주제"])[0]
#         ans = f"입력의 {qt}와 후보의 {dt}가 대응해 맥락상 높은 관련성을 보입니다."


#     if not ans:
#         # 완전 실패 시 백업
#         ans = "연구 대상과 접근 방식이 유사해 실제 활용 맥락에서 높은 연관성이 있습니다."
#     return ans


## 기존 모델(Recommender&cosine)

In [30]:
# -------------------- Recommender --------------------
class Recommender:
    def __init__(self, df: pd.DataFrame, backend: 'EmbeddingBackend|None' = None):  # 변경
        self.df = df.copy()
        self.df = self.df.rename(columns={c: c.lower() for c in self.df.columns})
        for col in ["title", "description", "url"]:
            if col not in self.df.columns:
                self.df[col] = ""

        self.df["__text__"] = (
            self.df["title"].map(safe_text) + " " + self.df["description"].map(safe_text)
        ).fillna("").str.strip()
        self.df = self.df[self.df["__text__"] != ""].reset_index(drop=True)

        self.backend = backend or get_backend()   # 외부에서 넘기면 재사용
        self.doc_vectors = None

    def fit(self):
        texts = self.df["__text__"].tolist()
        self.backend.fit(texts)
        self.doc_vectors = self.backend.encode(texts)

    def _cosine(self, A, B) -> np.ndarray:
        from sklearn.metrics.pairwise import cosine_similarity
        return cosine_similarity(A, B)[0]

    def encode_query_weighted(self, title: str, desc: str, w_title: float, w_desc: float):
        """제목/설명 임베딩을 같은 백엔드로 구해 가중합 → 질의 벡터"""
        t = safe_text(title)
        d = safe_text(desc)

        vt = self.backend.encode([t])
        vd = self.backend.encode([d])

        try:
            # scipy.sparse 지원(TF-IDF)
            from scipy import sparse
            if sparse.issparse(vt):
                q = vt.multiply(w_title) + vd.multiply(w_desc)
            else:
                q = vt * w_title + vd * w_desc
        except Exception:
            q = vt * w_title + vd * w_desc
        return q

    def recommend(self, title: str, desc: str, topk: int = 5) -> pd.DataFrame:
        q_vec = self.encode_query_weighted(title, desc, W_TITLE, W_DESC)
        sims = self._cosine(q_vec, self.doc_vectors)
        top_idx = np.argsort(-sims)[:topk]

        rows = []
        top_sims = sims[top_idx]
        for i in top_idx:
            row = self.df.iloc[i]
            score = float(sims[i])

            # 레벨 라벨링 유지
            level = label_by_ranked_score(top_sims, np.where(top_idx == i)[0][0])

            # 추천 사유: 추출형 한 문장으로 교체
            if USE_EXTRACTIVE_REASON:
                reason = extractive_reason(
                    title, desc,
                    safe_text(row.get("title","")),
                    safe_text(row.get("description","")),
                    self.backend,
                    max_chars=100,
                )
            else:
                # (옵션) 예전 방식 유지하려면 여기만 남겨두세요.
                if TOK_KIND == "skip":
                    reason_query = f"{safe_text(title)} {safe_text(desc)}"
                    reason_doc   = f"{safe_text(row.get('title',''))} {safe_text(row.get('description',''))}"
                    mode = "skip"
                else:
                    reason_query = tokenize(f"{safe_text(title)} {safe_text(desc)}")
                    reason_doc   = tokenize(row["__text__"])
                    mode = TOK_KIND
                reason = build_reason(reason_query, reason_doc, mode=mode)

            rows.append({
                "rank": len(rows) + 1,
                "title": row.get("title", ""),
                "description": row.get("description", ""),
                "url": row.get("url", ""),
                "score": round(score, 4),
                "reason": reason,
                "level": level,
            })
        return pd.DataFrame(rows)

def load_df(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # 아주 간단한 안전장치
    df = df.rename(columns={c: c.lower() for c in df.columns})
    for col in ["title", "description", "url"]:
        if col not in df.columns:
            df[col] = ""
    return df

def run_recommendation_for_corpus(df: pd.DataFrame, backend, title: str, desc: str, topk: int) -> pd.DataFrame:
    rec = Recommender(df, backend=backend)   # 같은 백엔드 재사용
    rec.fit()
    return rec.recommend(title, desc, topk=topk)


## Re-ranking

In [31]:
# === Multistage Re-ranking (BM25 → Dense → Cross-Encoder) ===
import numpy as np, pandas as pd, re, os
from typing import Dict, List, Tuple
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# ---------- Config ----------
# 단계별 후보 수
TOPN_BM25 = 200   # BM25 1차 후보
M_DENSE   = 60    # Dense 재스코어 후 유지
L_CE      = 15    # Cross-Encoder 재랭킹 대상
K_FINAL   = 5     # 최종 Top-K (3~5 권장)

# 점수 결합 가중치 (초기값 제안)
ALPHA = 0.35   # BM25 비중
BETA  = 0.65   # Dense 비중
GAMMA = 0.55   # (BM25+Dense) vs CE 비중

# BM25 필드 가중치 (필드 없으면 자동 무시)
FIELD_WEIGHTS = {"title": 2.0, "keywords": 1.6, "description": 1.0, "org": 0.6, "doi": 0.2}

# Cross-Encoder 사용 여부 및 모델 (다국어 추천)
USE_CE = True
CE_MODEL = "models/bge-reranker-v2-m3"  

# 다국어 이중 쿼리 가중(ko 우선)
W_LANG = 0.6   # q* = normalize(W_LANG*q_ko + (1-W_LANG)*q_en)

# ---------- Lightweight tokenizer ----------
_rx = re.compile(r"[가-힣A-Za-z0-9]+")
def lite_tokens(s: str) -> List[str]:
    return [t.lower() for t in _rx.findall(s or "") if len(t) > 1]

# ---------- Robust normalization ----------
def robust_minmax(x: np.ndarray, lo=5, hi=95) -> np.ndarray:
    if len(x) == 0:
        return x
    a, b = np.percentile(x, lo), np.percentile(x, hi)
    denom = max(1e-6, b - a)
    return np.clip((x - a) / denom, 0, 1)

# ---------- BM25 index per field ----------
class WeightedBM25:
    """각 필드별 BM25를 만들고 가중합으로 점수를 계산"""
    def __init__(self, df: pd.DataFrame, fields: Dict[str, float]):
        self.fields = {}
        for f, w in fields.items():
            if f in df.columns:
                docs = [lite_tokens(safe_text(x)) for x in df[f].fillna("").astype(str).tolist()]
                if sum(len(d) for d in docs) > 0:
                    self.fields[f] = (BM25Okapi(docs), w)
        self.n_docs = len(df)

    def score(self, query_tokens: List[str]) -> np.ndarray:
        scores = np.zeros(self.n_docs, dtype=float)
        for f, (bm25, w) in self.fields.items():
            scores += w * bm25.get_scores(query_tokens)
        return scores

# ---------- Dense(임베딩) 준비 ----------
def compose_dense_text(row: pd.Series) -> str:
    # title [SEP] top-8 keywords [SEP] short_desc(최대 300자)
    title = safe_text(row.get("title", ""))
    kws   = safe_text(row.get("keywords", ""))
    if isinstance(kws, list): kws = ", ".join(kws)
    kws = ", ".join(kws.split(",")[:8])
    desc  = safe_text(row.get("description", ""))[:300]
    return f"{title} [SEP] {kws} [SEP] {desc}".strip()

def build_dense_matrix(df: pd.DataFrame, backend) -> Tuple[List[str], np.ndarray]:
    texts = [compose_dense_text(r) for _, r in df.iterrows()]
    vecs  = backend.encode(texts)  # SBERTBackend.normalize_embeddings=True 권장
    return texts, vecs

def combine_query_vec(backend, ko_query: str, en_query: str | None) -> np.ndarray:
    q_ko = backend.encode([ko_query])[0]
    if en_query:
        q_en = backend.encode([en_query])[0]
        q = W_LANG * q_ko + (1 - W_LANG) * q_en
        q = q / (np.linalg.norm(q) + 1e-12)
        return q
    return q_ko

_ce_model_cache = None
def ce_predict_pairs(pairs: List[Tuple[str, str]]) -> np.ndarray:
    global _ce_model_cache
    if not USE_CE:
        return np.zeros(len(pairs), dtype=float)

    if _ce_model_cache is None:
        from sentence_transformers import CrossEncoder
        import torch
        dev = "cuda" if torch.cuda.is_available() else "cpu"

        def _load(model_kwargs=None):
            return CrossEncoder(
                CE_MODEL, device=dev, max_length=512,
                **({"model_kwargs": model_kwargs} if model_kwargs else {})
            )

        try:
            # 1) SDPA 시도 (가능하면 속도 이점)
            _ce_model_cache = _load({"attn_implementation": "sdpa"})
        except Exception:
            # 2) SDPA 미지원 시 eager로 폴백
            _ce_model_cache = _load({"attn_implementation": "eager"})

    return _ce_model_cache.predict(pairs)

# ---------- 파이프라인 본체 ---------- (파이프라인 전체 흐름, 점수 결합 로직)
def multistage_recommend(
    title_ko: str, desc_ko: str,
    papers_df: pd.DataFrame, datasets_df: pd.DataFrame,
    backend,
    en_title: str | None = None, en_desc: str | None = None,
    topk: int = K_FINAL
) -> pd.DataFrame:

    # ★ 캐시 보장
    _ensure_indexes_and_dense(papers_df, datasets_df, backend)

    # 0) 쿼리 문자열
    q_ko = (safe_text(title_ko) + " " + safe_text(desc_ko)).strip()
    q_en = (safe_text(en_title) + " " + safe_text(en_desc)).strip() if (en_title or en_desc) else None

    # 1) BM25 (캐시 사용)
    bm25_p, bm25_d = _BM25_P, _BM25_D
    q_tokens = lite_tokens(q_ko) + (lite_tokens(q_en) if q_en else [])
    b_p = bm25_p.score(q_tokens); idx_p = np.argsort(-b_p)[:min(TOPN_BM25, len(papers_df))]
    b_d = bm25_d.score(q_tokens); idx_d = np.argsort(-b_d)[:min(TOPN_BM25, len(datasets_df))]

    # 2) Dense (캐시된 임베딩에서 후보만 참조)
    q_vec = combine_query_vec(backend, q_ko, q_en)
    s_p_dense = _P_DENSE_VECS[idx_p] @ q_vec
    s_d_dense = _D_DENSE_VECS[idx_d] @ q_vec

    cand_p = pd.DataFrame({"src":"paper","idx":idx_p, "bm25":b_p[idx_p], "dense":s_p_dense})
    cand_d = pd.DataFrame({"src":"dataset","idx":idx_d, "bm25":b_d[idx_d], "dense":s_d_dense})
    cand   = pd.concat([cand_p, cand_d], ignore_index=True).sort_values("dense", ascending=False)
    cand   = cand.head(min(M_DENSE, len(cand))).reset_index(drop=True)

    # 2.5) 정규화/기본점수
    cand["bm25_n"] = robust_minmax(cand["bm25"].to_numpy())
    cand["dense_n"] = robust_minmax(cand["dense"].to_numpy())
    cand["s_base"]  = ALPHA * cand["bm25_n"] + BETA * cand["dense_n"]

    # 3) CE 재랭킹
    cand_L = cand.head(min(L_CE, len(cand))).copy()
    q_text = q_en if q_en else q_ko
    pairs = [
        (q_text, (_P_DENSE_TEXTS if src == "paper" else _D_DENSE_TEXTS)[int(i)])
        for src, i in zip(cand_L["src"], cand_L["idx"])
    ]
    ce_scores = ce_predict_pairs(pairs) if len(pairs) else np.array([])
    cand_L["ce"] = ce_scores if len(ce_scores) else 0.0

    # 4) 점수 결합
    cand["final"] = cand["s_base"].to_numpy()
    if len(cand_L):
        cand_L["ce_n"] = robust_minmax(cand_L["ce"].to_numpy())
        base_vals = cand.loc[cand_L.index, "s_base"].to_numpy()
        cand.loc[cand_L.index, "final"] = GAMMA * base_vals + (1 - GAMMA) * cand_L["ce_n"].to_numpy()

    # 레벨링
    base_for_levels = cand.head(min(L_CE, len(cand)))
    p50, p75, p90 = np.percentile(base_for_levels["final"].to_numpy(), [50, 75, 90])
    def to_level(x: float) -> str:
        if x >= p90: return "강추"
        if x >= p75: return "추천"
        if x >= p50: return "참고"
        return "보류"
    cand["level"] = cand["final"].apply(to_level)

    # 5) Top-K
    top = cand.sort_values("final", ascending=False).head(min(topk, len(cand))).copy()

    # 6) 표 생성 (추천사유는 초경량 버전 권장)
    rows = []
    for _, r in top.iterrows():
        src, i = r["src"], int(r["idx"])
        row = (papers_df.iloc[i] if src == "paper" else datasets_df.iloc[i])

        # 임베딩 호출 없는 초경량 사유 (강력 추천)
        reason = extractive_reason(
            USER_TITLE, USER_DESC,
            safe_text(row.get("title","")),
            safe_text(row.get("description","")),
            backend,
            max_chars=100,
        )

        rows.append({
            "구분": "thesis" if src=="paper" else "dataset",
            "제목": safe_text(row.get("title","")),
            "설명": safe_text(row.get("description","")),
            "점수": round(float(r["final"]), 4),
            "추천 사유": reason,
            "Level": r.get("level", "참고"),
            "URL":  safe_text(row.get("url","")),
        })
    return pd.DataFrame(rows)

## 입력 부분

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../Clarify")))

from clarify_utils import ClarifyModule

In [ ]:
# ==== 질의 입력 & 실행 ====
print("=== 질의 입력(다단계 재랭킹) ===")
USER_TITLE = input("제목을 입력하세요: ").strip()
USER_DESC  = input("설명을 입력하세요(빈칸 가능): ").strip()

if not USER_TITLE and not USER_DESC:
    raise ValueError("제목/설명 중 하나는 반드시 입력해야 합니다.")

# ==== Clarify 통합 ====
from clarify_utils import ClarifyModule
clarifier = ClarifyModule()

# Clarify를 통해 한국어든 영어든 → 명확한 영어 주제로 변환
EN_TITLE = clarifier.clarify(USER_TITLE) if USER_TITLE else None
EN_DESC  = clarifier.clarify(USER_DESC) if USER_DESC else None

print("\n[Clarify 결과]")
print("제목 Clarified →", EN_TITLE)
print("설명 Clarified →", EN_DESC)

# ==== 백엔드 준비 및 실행 ====
backend = get_backend()

papers_df   = load_df(PAPERS_CSV)
datasets_df = load_df(DATASETS_CSV)

final_df = multistage_recommend(
    title_ko=USER_TITLE, desc_ko=USER_DESC,
    papers_df=papers_df, datasets_df=datasets_df,
    backend=backend,
    en_title=EN_TITLE, en_desc=EN_DESC,
    topk=max(3, min(K_FINAL, 5))
)

from IPython.display import display
print(f"\n[논문+데이터셋 통합 TOP{len(final_df)} — 다단계 재랭킹]")
display(final_df)

final_df.to_csv("추천_통합_다단계.csv", index=False, encoding="utf-8-sig")
print("\n저장 완료:", os.path.abspath("추천_통합_다단계.csv"))

=== 질의 입력(다단계 재랭킹) ===


제목을 입력하세요:  The Effect of Self-Construal on Continued Intention of Conversational AI Agent : The Boundary Condition of Mind Perception
설명을 입력하세요(빈칸 가능):  상호의존적 자아가 독립적 자아보다 대화형 AI의 지속 사용 의도가 높음을 시나리오 실험으로 확인됐다. 사회불안에서 고독감으로 이어지는 연쇄 매개와 마음 지각이 사용 의도를 강화하는 조절 요인임을 함께 검증한다.



[논문+데이터셋 통합 TOP5 — 다단계 재랭킹]


,구분,제목,설명,점수,추천 사유,Level,URL
0,thesis,AI 스피커 이용의도에 관한 연구 : 개인의 커뮤니케이션 성향과 AI 스피커에 대한...,본 연구는 개인의 커뮤니케이션 관련 성향과 AI 스피커에 대한 인식이 AI 스피커 ...,0.8509,"사회불안은 AI 스피커 이용의도와 정 적 관계를, 커뮤니케이션 회피성향은 부 적 관...",강추,http://click.ndsl.kr/servlet/OpenAPIDetailView...
1,thesis,생성형 AI와 대중의 인식: 이미지 생성형 AI에 대한 설문조사 결과,"본 연구의 목적은 이미지 생성형 AI(예: 미드저니, 달리)에 대한 대중의 인식, ...",0.7429,"본 연구의 목적은 이미지 생성형 AI 에 대한 대중의 인식, 사용 경험 및 만족도를...",강추,http://click.ndsl.kr/servlet/OpenAPIDetailView...
2,thesis,AI 수용의도가 적응 수행에 미치는 영향 : 생성형 AI 리터러시의 매개효과,본 연구의 목적은 최근 각광받고 있는 생성형 AI 리터러시에 대한 척도를 개발하는 ...,0.6359,"또한, 생성형 AI 수용 의도가 높은 개인이 생성형 AI 리터러시를 통해 적응 수행...",추천,http://click.ndsl.kr/servlet/OpenAPIDetailView...
3,thesis,ETRI AI 실행전략 6: 산업&#183;공공 AI 활용기술 연구개발 및 적용,As the development of artificial intelligence ...,0.5578,As the development of artificial intelligence ...,추천,http://click.ndsl.kr/servlet/OpenAPIDetailView...
4,thesis,The Direction of AI Classes using AI Education...,본 연구는 AI 플랫폼을 활용한 인공지능 수업에서 효과적인 내용과 방법을 제시하고자...,0.5560,AI 플랫폼을 활용한 인공지능 수업에서 효과적인 내용과 방법을 제시하고자 하였다.,추천,http://click.ndsl.kr/servlet/OpenAPIDetailView...



저장 완료: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\추천_통합_다단계.csv


# 검증

## 쿼리 샘플 뽑기 + 주석 시트 만들기(정밀)

In [ ]:
# === A) 쿼리 샘플 뽑기 + 주석 시트 생성 ===
# - 코퍼스에서 N_Q개 쿼리를 샘플링하고
# - 각 쿼리별 우리 파이프라인으로 후보 TOPK_CAND를 생성해서
#   eval_annotation_sheet.csv 를 만든다.

import numpy as np, pandas as pd, hashlib, os

N_Q = 50           # 평가용 쿼리 수 (50~100 권장)
TOPK_CAND = 30     # 쿼리당 후보 수 (20~30 권장)
ANNOT_PATH = "eval_annotation_sheet.csv"

def safe_text(x):
    return "" if pd.isna(x) else str(x)

def row_to_id(src: str, title: str, url: str) -> str:
    """doc_id 통일: URL이 있으면 URL, 없으면 src+제목 hash"""
    url = safe_text(url)
    if url:
        return url
    h = hashlib.md5(safe_text(title).encode("utf-8")).hexdigest()[:10]
    return f"{src}:{h}"

def make_queries_from_corpus(papers_df, datasets_df, n=50, seed=42):
    # 각 코퍼스에서 절반씩 샘플. 원하면 비율 조정 가능
    rng = np.random.default_rng(seed)
    n_p = min(n//2, len(papers_df))
    n_d = min(n - n_p, len(datasets_df))
    sp = papers_df.sample(n_p, random_state=seed) if n_p>0 else papers_df.head(0)
    sd = datasets_df.sample(n_d, random_state=seed) if n_d>0 else datasets_df.head(0)
    rows = []

    # 논문 쪽
    for _, r in sp.iterrows():
        rows.append({
            "qid": f"Q{len(rows)+1:03d}",
            "q_title": safe_text(r.get("title","")),
            "q_desc":  safe_text(r.get("description","")),
            "q_src":   "thesis",
            "q_url":   safe_text(r.get("url","")),
        })
    # 데이터셋 쪽
    for _, r in sd.iterrows():
        rows.append({
            "qid": f"Q{len(rows)+1:03d}",
            "q_title": safe_text(r.get("title","")),
            "q_desc":  safe_text(r.get("description","")),
            "q_src":   "dataset",
            "q_url":   safe_text(r.get("url","")),
        })
    return pd.DataFrame(rows)

def build_annotation_sheet(queries_df, topk_cand=30):
    ann_rows = []
    for _, q in queries_df.iterrows():
        df = multistage_recommend(
            title_ko=q["q_title"], desc_ko=q["q_desc"],
            papers_df=papers_df, datasets_df=datasets_df,
            backend=backend, en_title=None, en_desc=None,
            topk=topk_cand
        )
        for _, r in df.iterrows():
            src = safe_text(r.get("구분",""))  # "thesis"/"dataset"
            title = safe_text(r.get("제목","") or r.get("title",""))
            desc  = safe_text(r.get("설명","") or r.get("description",""))
            url   = safe_text(r.get("URL","") or r.get("url",""))
            doc_id = row_to_id(src, title, url)

            # 자기 자신(쿼리의 원문)과 동일 URL이면 제외
            if q["q_url"] and url and (q["q_url"] == url):
                continue

            ann_rows.append({
                "qid": q["qid"],
                "q_title": q["q_title"],
                "q_desc":  q["q_desc"],
                "cand_id": doc_id,         # 평가에서 사용할 고유 id
                "cand_url": url,           # 사람이 보기 편하도록 URL도 보관
                "cand_title": title,
                "cand_desc": desc,
                "cand_source": src,        # thesis/dataset
                "baseline_score": float(r.get("점수", 0.0)),  # 파이프라인 점수
                "rel": ""                  # (사람 라벨 대신 다음 셀에서 자동 채움)
            })
    return pd.DataFrame(ann_rows)

# 실행
queries_df = make_queries_from_corpus(papers_df, datasets_df, n=N_Q)
ann = build_annotation_sheet(queries_df, topk_cand=TOPK_CAND)
ann.to_csv(ANNOT_PATH, index=False, encoding="utf-8-sig")
print("주석 시트 저장:", os.path.abspath(ANNOT_PATH))
display(ann.head(3))


주석 시트 저장: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\eval_annotation_sheet.csv


,qid,q_title,q_desc,cand_id,cand_url,cand_title,cand_desc,cand_source,baseline_score,rel
0,Q001,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,기업의 빅데이터 활용 유형별 적용 방안 및 효과 분석,지난 수년 간 스마트폰과 같은 스마트 기기의 빠른 확산과 함께 인터넷과 소셜네트워크...,thesis,0.8157,
1,Q001,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,빅데이터 분석기법을 통한 N-스크린 서비스 활용에 영향을 미치는 요인 분석 : 로지...,스마트 디바이스의 확산으로 인해 정보통신 서비스의 다양성과 양이 크게 증가하고 있다...,thesis,0.7999,
2,Q001,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,빅데이터 분석을 통한 사용자 관점의 이슈 클러스터링,급증하는 빅 데이터 분석은 웹 3.0과 스마트 기기의 보편화에 대응하기 위한 기업의...,thesis,0.7736,


## 정밀 모드 CE/Hybrid 자동 라벨링

In [ ]:
# === B) 정밀 모드 자동 라벨링 (CE / Hybrid) ===
# - A셀에서 만든 eval_annotation_sheet.csv를 읽어서
# - CE 또는 Hybrid 점수로 퍼센타일 컷 → rel(2/1/0) 채움
# - eval_gold.csv 생성

import os, re, numpy as np, pandas as pd, torch
from sentence_transformers import CrossEncoder

ANNOT_PATH = "eval_annotation_sheet.csv"
OUT_ANN    = "eval_annotation_sheet.auto_labeled.csv"
OUT_GOLD   = "eval_gold.csv"

LABEL_MODE = "ce"                 # "ce" | "hybrid"
CE_MODEL   = "BAAI/bge-reranker-v2-m3"
W_CE, W_BASE = 0.55, 0.45         # hybrid 가중치
P_HIGH, P_MID = 90, 75            # p90→2, p75→1
ABS_STRONG = None                 # 예: 0.70
ABS_WEAK   = None                 # 예: 0.55

def safe_text(x): return "" if pd.isna(x) else str(x)
def robust_minmax(x, lo=5, hi=95):
    x = np.asarray(x, dtype=float)
    if len(x)==0: return x
    a,b = np.percentile(x, [lo,hi]); denom = max(1e-6, b-a)
    return np.clip((x-a)/denom, 0, 1)

# CE 로더 (SDPA→Eager 폴백)
_ce_model_cache = None
def ce_score_pairs(pairs, model_name=CE_MODEL):
    global _ce_model_cache
    if _ce_model_cache is None:
        dev = "cuda" if torch.cuda.is_available() else "cpu"
        def _load(model_kwargs=None):
            return CrossEncoder(model_name, device=dev, max_length=512,
                                **({"model_kwargs": model_kwargs} if model_kwargs else {}))
        try:
            _ce_model_cache = _load({"attn_implementation": "sdpa"})
        except Exception:
            _ce_model_cache = _load({"attn_implementation": "eager"})
    return _ce_model_cache.predict(pairs)

# 로드 & 점수 계산
ann = pd.read_csv(ANNOT_PATH)
pairs = []
for _, r in ann.iterrows():
    q_text = f"{safe_text(r['q_title'])} [SEP] {safe_text(r['q_desc'])}"
    d_text = f"{safe_text(r['cand_title'])} [SEP] {safe_text(r['cand_desc'])}"
    pairs.append((q_text, d_text))
ce_scores = ce_score_pairs(pairs) if len(pairs) else np.array([])
ann["ce_score"] = ce_scores if len(pairs) else 0.0
ann["baseline_score"] = pd.to_numeric(ann.get("baseline_score", 0.0), errors="coerce").fillna(0.0)

def make_silver_score(g):
    if LABEL_MODE == "ce":
        return g["ce_score"].to_numpy(float)
    elif LABEL_MODE == "hybrid":
        return 0.55*robust_minmax(g["ce_score"].to_numpy(float)) + \
               0.45*robust_minmax(g["baseline_score"].to_numpy(float))
    else:
        raise ValueError("LABEL_MODE must be 'ce' or 'hybrid'.")

def assign_rel_per_query(g):
    s = make_silver_score(g)
    if len(s) >= 3:
        q_high, q_mid = np.percentile(s, [P_HIGH, P_MID])
    else:
        q_high, q_mid = (np.max(s) if len(s) else 0.0, np.median(s) if len(s) else 0.0)
    rel = np.zeros(len(s), dtype=int)
    rel[s >= q_mid] = 1
    rel[s >= q_high] = 2
    if ABS_STRONG is not None:
        rel[(s >= q_high) & (s < ABS_STRONG)] = 1
    if ABS_WEAK is not None:
        rel[(s < q_mid) & (s >= ABS_WEAK)] = 1
    g = g.copy()
    g["silver_score"], g["rel"], g["q_high"], g["q_mid"] = s, rel, q_high, q_mid
    return g

groups = []
for qid, g in ann.groupby("qid", sort=False):
    gg = assign_rel_per_query(g.drop(columns=["qid"], errors="ignore"))  # 그룹 키 제외하고 처리
    gg["qid"] = qid                                                      # 처리 후 다시 붙이기
    groups.append(gg)

ann_lab = pd.concat(groups, ignore_index=True)
ann_lab.to_csv(OUT_ANN, index=False, encoding="utf-8-sig")
print("자동 라벨 시트 저장:", os.path.abspath(OUT_ANN))

# gold 압축 (doc_id = cand_id 사용!)
gold_rows = []
for qid, grp in ann_lab.groupby("qid"):
    pos = grp[grp["rel"] > 0]
    if len(pos)==0: 
        continue
    urls = [safe_text(u) for u in pos["cand_id"].tolist()]  # ← 평가에서 사용할 doc_id
    rels = [int(x) for x in pos["rel"].tolist()]
    q_title = safe_text(grp.iloc[0]["q_title"])
    q_desc  = safe_text(grp.iloc[0]["q_desc"])
    gold_rows.append({
        "qid": qid,
        "title": q_title,
        "desc": q_desc,
        "gold_urls": ",".join(urls),
        "gold_rels": ",".join(map(str, rels)),
    })

pd.DataFrame(gold_rows).to_csv(OUT_GOLD, index=False, encoding="utf-8-sig")
print("gold 저장:", os.path.abspath(OUT_GOLD))
display(ann_lab.head(3))

자동 라벨 시트 저장: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\eval_annotation_sheet.auto_labeled.csv
gold 저장: C:\Users\ogod3\바탕 화면\유섭 - desktop\취준&대학원\DATA&AI 공모전\eval_gold.csv


,q_title,q_desc,cand_id,cand_url,cand_title,cand_desc,cand_source,baseline_score,rel,ce_score,silver_score,q_high,q_mid,qid
0,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,기업의 빅데이터 활용 유형별 적용 방안 및 효과 분석,지난 수년 간 스마트폰과 같은 스마트 기기의 빠른 확산과 함께 인터넷과 소셜네트워크...,thesis,0.8157,0,0.002264,0.002264,0.017025,0.005185,Q001
1,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,빅데이터 분석기법을 통한 N-스크린 서비스 활용에 영향을 미치는 요인 분석 : 로지...,스마트 디바이스의 확산으로 인해 정보통신 서비스의 다양성과 양이 크게 증가하고 있다...,thesis,0.7999,0,0.000605,0.000605,0.017025,0.005185,Q001
2,빅데이터 기반의 로그데이터 분석을 통한 시스템 장애 감지기법 연구,최근 인터넷 서비스의 증가와 다양화로 인해 서비스 로그들은 기하급수적으로 증가하고 ...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,http://click.ndsl.kr/servlet/OpenAPIDetailView...,빅데이터 분석을 통한 사용자 관점의 이슈 클러스터링,급증하는 빅 데이터 분석은 웹 3.0과 스마트 기기의 보편화에 대응하기 위한 기업의...,thesis,0.7736,0,0.000691,0.000691,0.017025,0.005185,Q001


## 평가 유틸 (nDCG/MRR/Recall)

In [15]:
# === C) 평가 유틸 ===
import numpy as np

def dcg_at_k(rels, k=10):
    rels = np.asfarray(rels)[:k]
    if rels.size:
        discounts = 1.0 / np.log2(np.arange(2, rels.size + 2))
        return float(np.sum(rels * discounts))
    return 0.0

def ndcg_at_k(rels, k=10):
    dcg = dcg_at_k(rels, k)
    idcg = dcg_at_k(sorted(rels, reverse=True), k)
    return float(dcg / idcg) if idcg > 0 else 0.0

def mrr_at_k(rels, k=10):
    rels = np.asfarray(rels)[:k]
    hits = np.where(rels > 0)[0]
    return float(1.0 / (hits[0] + 1)) if len(hits) else 0.0

def recall_at_k(rels, num_pos_total, k=10):
    if num_pos_total <= 0: return 0.0
    rels = np.asfarray(rels)[:k]
    return float(np.sum(rels > 0) / num_pos_total)

def evaluate(queries, retrieve_fn, gold, k=10):
    ndcgs, mrrs, recalls = [], [], []
    for q in queries:
        qid = q["qid"]
        results = retrieve_fn(q, topk=k)
        rels = [gold.get(qid, {}).get(doc_id, 0) for doc_id, _ in results]
        num_pos = sum(1 for v in gold.get(qid, {}).values() if v > 0)
        ndcgs.append(ndcg_at_k(rels, k))
        mrrs.append(mrr_at_k(rels, k))
        recalls.append(recall_at_k(rels, num_pos, k))
    return {
        f"nDCG@{k}": float(np.mean(ndcgs)) if ndcgs else 0.0,
        f"MRR@{k}": float(np.mean(mrrs)) if mrrs else 0.0,
        f"Recall@{k}": float(np.mean(recalls)) if recalls else 0.0,
        "N": len(queries),
    }

## 평가 실행 (우리 파이프라인 ↔ gold 연결)

In [44]:
# === D) 평가 실행 ===
import pandas as pd, hashlib

# gold 로드 → queries 리스트 & gold dict 만들기
gdf = pd.read_csv("eval_gold.csv")
queries = gdf[["qid","title","desc"]].drop_duplicates().rename(
    columns={"title":"title","desc":"desc"}
).to_dict("records")

# gold dict: {qid: {doc_id: rel}}
gold = {}
for _, r in gdf.iterrows():
    urls = [u.strip() for u in str(r["gold_urls"]).split(",") if u.strip()]
    rels = [int(x) for x in str(r["gold_rels"]).split(",") if str(x).strip().isdigit()]
    if rels and len(rels) != len(urls):
        rels = [1]*len(urls)
    if not rels:
        rels = [1]*len(urls)
    gold[r["qid"]] = {u: rels[i] for i,u in enumerate(urls)}

# doc_id 생성 로직을 annotation 시트와 동일하게 맞춘다
def row_to_id(src: str, title: str, url: str) -> str:
    url = str(url or "")
    if url:
        return url
    h = hashlib.md5(str(title or "").encode("utf-8")).hexdigest()[:10]
    return f"{src}:{h}"

# 우리 파이프라인 호출 → doc_id 리스트 반환
def retrieve_full(q, topk=10):
    df = multistage_recommend(
        title_ko=q["title"], desc_ko=q["desc"],
        papers_df=papers_df, datasets_df=datasets_df,
        backend=backend, en_title=None, en_desc=None, topk=topk
    )
    out = []
    for _, r in df.iterrows():
        src   = str(r.get("구분",""))
        title = str(r.get("제목","") or r.get("title",""))
        url   = str(r.get("URL","") or r.get("url",""))
        doc_id = row_to_id(src, title, url)
        out.append((doc_id, float(r.get("점수", 0.0))))
    return out

report = evaluate(
    [{"qid":q["qid"], "title":q["title"], "desc":q["desc"]} for q in queries],
    retrieve_full, gold, k=10
)
print(report)

{'nDCG@10': 0.6489751237263062, 'MRR@10': 0.4725555555555555, 'Recall@10': 0.5375, 'N': 50}


## 빠른 평가용(간소화) - but 평가 성능 떨어짐

In [11]:
# === Cache once ===
try:
    bm25_p, bm25_d, vecs_p, vecs_d
except NameError:
    bm25_p = WeightedBM25(papers_df,   FIELD_WEIGHTS)
    bm25_d = WeightedBM25(datasets_df, FIELD_WEIGHTS)
    texts_p = [compose_dense_text(r) for _, r in papers_df.iterrows()]
    texts_d = [compose_dense_text(r) for _, r in datasets_df.iterrows()]
    vecs_p  = backend.encode(texts_p)   # normalize=True → 코사인=내적
    vecs_d  = backend.encode(texts_d)

def _robust_minmax(x, lo=5, hi=95):
    import numpy as np
    a,b = np.percentile(x, [lo,hi]); denom = max(1e-6, b-a)
    return np.clip((x-a)/denom, 0, 1)

In [12]:
import numpy as np, pandas as pd, hashlib

def row_to_id(src: str, title: str, url: str) -> str:
    url = str(url or "")
    if url: return url
    h = hashlib.md5(str(title or "").encode("utf-8")).hexdigest()[:10]
    return f"{src}:{h}"

# 2-1) BM25-only
def retrieve_bm25_only(q, topk=10):
    q_ko = f"{q['title']} {q['desc']}".strip()
    toks = lite_tokens(q_ko)
    b_p = bm25_p.score(toks); b_d = bm25_d.score(toks)

    idx_p = np.argpartition(-b_p, min(TOPN_BM25, len(b_p))-1)[:min(TOPN_BM25, len(b_p))]
    idx_d = np.argpartition(-b_d, min(TOPN_BM25, len(b_d))-1)[:min(TOPN_BM25, len(b_d))]
    cand_p = pd.DataFrame({"src":"thesis","idx":idx_p, "bm25":b_p[idx_p]})
    cand_d = pd.DataFrame({"src":"dataset","idx":idx_d, "bm25":b_d[idx_d]})
    cand   = pd.concat([cand_p, cand_d], ignore_index=True)

    cand["s"] = _robust_minmax(cand["bm25"].to_numpy())  # 점수는 BM25 정규화
    cand = cand.sort_values("s", ascending=False).head(topk)

    out = []
    for _, r in cand.iterrows():
        i, src = int(r["idx"]), r["src"]
        row = (papers_df.iloc[i] if src=="thesis" else datasets_df.iloc[i])
        out.append((row_to_id(src, row.get("title",""), row.get("url","")), float(r["s"])))
    return out

# 2-2) Dense-only (전 문서 벡터와 내적 → 상위 선별)
def retrieve_dense_only(q, topk=10, N_DENSE_POOL=500):
    q_ko = f"{q['title']} {q['desc']}".strip()
    q_vec = combine_query_vec(backend, q_ko, None)

    s_p = vecs_p @ q_vec
    s_d = vecs_d @ q_vec
    kp  = min(N_DENSE_POOL, len(s_p))
    kd  = min(N_DENSE_POOL, len(s_d))
    idx_p = np.argpartition(-s_p, kp-1)[:kp]
    idx_d = np.argpartition(-s_d, kd-1)[:kd]
    cand_p = pd.DataFrame({"src":"thesis","idx":idx_p, "dense":s_p[idx_p]})
    cand_d = pd.DataFrame({"src":"dataset","idx":idx_d, "dense":s_d[idx_d]})
    cand   = pd.concat([cand_p, cand_d], ignore_index=True)

    cand["s"] = _robust_minmax(cand["dense"].to_numpy())
    cand = cand.sort_values("s", ascending=False).head(topk)
    out = []
    for _, r in cand.iterrows():
        i, src = int(r["idx"]), r["src"]
        row = (papers_df.iloc[i] if src=="thesis" else datasets_df.iloc[i])
        out.append((row_to_id(src, row.get("title",""), row.get("url","")), float(r["s"])))
    return out

# 2-3) BM25→Dense (현재 “빠른” 기본형: s_base = α*BM25n + β*Densen)
def retrieve_bm25_dense(q, topk=10):
    q_ko = f"{q['title']} {q['desc']}".strip()
    toks = lite_tokens(q_ko)
    b_p = bm25_p.score(toks); b_d = bm25_d.score(toks)

    kbp = min(TOPN_BM25, len(b_p)); kbd = min(TOPN_BM25, len(b_d))
    idx_p = np.argpartition(-b_p, kbp-1)[:kbp]
    idx_d = np.argpartition(-b_d, kbd-1)[:kbd]

    q_vec = combine_query_vec(backend, q_ko, None)
    s_p = vecs_p[idx_p] @ q_vec
    s_d = vecs_d[idx_d] @ q_vec

    cand_p = pd.DataFrame({"src":"thesis","idx":idx_p, "bm25":b_p[idx_p], "dense":s_p})
    cand_d = pd.DataFrame({"src":"dataset","idx":idx_d, "bm25":b_d[idx_d], "dense":s_d})
    cand   = pd.concat([cand_p, cand_d], ignore_index=True)
    cand["bm25_n"]  = _robust_minmax(cand["bm25"].to_numpy())
    cand["dense_n"] = _robust_minmax(cand["dense"].to_numpy())
    cand["s_base"]  = ALPHA * cand["bm25_n"] + BETA * cand["dense_n"]

    cand = cand.sort_values("s_base", ascending=False).head(topk)
    out = []
    for _, r in cand.iterrows():
        i, src = int(r["idx"]), r["src"]
        row = (papers_df.iloc[i] if src=="thesis" else datasets_df.iloc[i])
        out.append((row_to_id(src, row.get("title",""), row.get("url","")), float(r["s_base"])))
    return out

# 2-4) BM25+Dense+CE (정밀, 느림: 작은 샘플에만)
from sentence_transformers import CrossEncoder
_ce_model = None
def retrieve_bm25_dense_ce(q, topk=10, L_CE_eval=30, CE_MODEL="BAAI/bge-reranker-v2-m3"):
    global _ce_model
    # 후보는 bm25_dense로 만들고 상위 L_CE_eval만 CE 재정렬
    base = retrieve_bm25_dense(q, topk=L_CE_eval)

    if _ce_model is None:
        import torch
        dev = "cuda" if torch.cuda.is_available() else "cpu"
        def _load(kwargs=None):
            return CrossEncoder(CE_MODEL, device=dev, max_length=512,
                                **({"model_kwargs": kwargs} if kwargs else {}))
        try:
            _ce_model = _load({"attn_implementation":"sdpa"})
        except Exception:
            _ce_model = _load({"attn_implementation":"eager"})

    # CE 입력쌍 만들기
    q_text = f"{q['title']} [SEP] {q['desc']}"
    pairs, rows = [], []
    for doc_id, _ in base:
        # doc_id를 다시 row로 역참조(빠르게 하려면 doc_id→row 인덱스 캐시 추천)
        # 여기선 간단히 URL 매칭으로 시도
        src, title_or_hash = doc_id.split(":", 1) if "://" not in doc_id else (None, None)
        # 안전한 버전: base 후보를 다시 multistage 없이 제목/URL로 찾아 텍스트 구성
        # 실용상 base 점수만으로도 상위는 잘 들어오므로 간단화:
        # (생략) → pairs에는 title/desc가 필요하므로 여기선 retrieve_bm25_dense를 좀 수정해야 하지만
        # 너가 이미 multistage_recommend 결과 프레임을 가지고 있다면 그걸 재사용하는 편이 더 깔끔해.
        pass  # ★ CE 정밀평가는 느리므로 소수 쿼리에서 multistage_recommend로 돌리는 편을 권장


In [13]:
def retrieve_union_bm25_dense(q, topk=10, N_BM25=400, N_DENSE=400):
    q_ko = f"{q['title']} {q['desc']}".strip()
    toks = lite_tokens(q_ko)
    b_p = bm25_p.score(toks); b_d = bm25_d.score(toks)

    kbp = min(N_BM25, len(b_p)); kbd = min(N_BM25, len(b_d))
    idx_bp = np.argpartition(-b_p, kbp-1)[:kbp]
    idx_bd = np.argpartition(-b_d, kbd-1)[:kbd]

    q_vec = combine_query_vec(backend, q_ko, None)
    s_all_p = vecs_p @ q_vec
    s_all_d = vecs_d @ q_vec
    kdp = min(N_DENSE, len(s_all_p)); kdd = min(N_DENSE, len(s_all_d))
    idx_dp = np.argpartition(-s_all_p, kdp-1)[:kdp]
    idx_dd = np.argpartition(-s_all_d, kdd-1)[:kdd]

    # 합집합 후보
    idx_p = np.unique(np.concatenate([idx_bp, idx_dp]))
    idx_d = np.unique(np.concatenate([idx_bd, idx_dd]))

    # 점수 계산
    cand_p = pd.DataFrame({
        "src":"thesis","idx":idx_p,
        "bm25":b_p[idx_p], "dense":s_all_p[idx_p]
    })
    cand_d = pd.DataFrame({
        "src":"dataset","idx":idx_d,
        "bm25":b_d[idx_d], "dense":s_all_d[idx_d]
    })
    cand = pd.concat([cand_p, cand_d], ignore_index=True)
    cand["bm25_n"]  = _robust_minmax(cand["bm25"].to_numpy())
    cand["dense_n"] = _robust_minmax(cand["dense"].to_numpy())
    cand["s_base"]  = ALPHA * cand["bm25_n"] + BETA * cand["dense_n"]

    cand = cand.sort_values("s_base", ascending=False).head(topk)

    out = []
    for _, r in cand.iterrows():
        i, src = int(r["idx"]), r["src"]
        row = (papers_df.iloc[i] if src=="thesis" else datasets_df.iloc[i])
        out.append((row_to_id(src, row.get("title",""), row.get("url","")), float(r["s_base"])))
    return out

In [19]:
# gold 로드 → queries 리스트 & gold dict 만들기
gdf = pd.read_csv("eval_gold.csv")
queries = gdf[["qid","title","desc"]].drop_duplicates().rename(
    columns={"title":"title","desc":"desc"}
).to_dict("records")

# gold dict: {qid: {doc_id: rel}}
gold = {}
for _, r in gdf.iterrows():
    urls = [u.strip() for u in str(r["gold_urls"]).split(",") if u.strip()]
    rels = [int(x) for x in str(r["gold_rels"]).split(",") if str(x).strip().isdigit()]
    if rels and len(rels) != len(urls):
        rels = [1]*len(urls)
    if not rels:
        rels = [1]*len(urls)
    gold[r["qid"]] = {u: rels[i] for i,u in enumerate(urls)}

In [20]:
runs = [
    ("BM25", retrieve_bm25_only),
    ("Dense", retrieve_dense_only),
    ("BM25+Dense", retrieve_bm25_dense),
    ("Recall++ (Union)", retrieve_union_bm25_dense),
    # ("BM25+Dense+CE (slow sample)", retrieve_bm25_dense_ce),  # 샘플 소수에만 권장
]
for name, fn in runs:
    rep = evaluate(
        [{"qid":q["qid"], "title":q["title"], "desc":q["desc"]} for q in queries],
        fn, gold, k=10
    )
    print(name, rep)

BM25 {'nDCG@10': 0.47664260661637575, 'MRR@10': 0.4027698412698413, 'Recall@10': 0.245, 'N': 50}
Dense {'nDCG@10': 0.4205274025639276, 'MRR@10': 0.3129047619047619, 'Recall@10': 0.2075, 'N': 50}
BM25+Dense {'nDCG@10': 0.5987783055813736, 'MRR@10': 0.524079365079365, 'Recall@10': 0.41, 'N': 50}
Recall++ (Union) {'nDCG@10': 0.5253263317774505, 'MRR@10': 0.4214126984126984, 'Recall@10': 0.3375, 'N': 50}
